In [11]:
import json
import re
import emoji

# =========================================================
# ⚙️ 参数
# =========================================================
INPUT_FILE = "recipe_corpus_full.json"     # JSONL
OUTPUT_TEXT_FILE = "recipes_text.txt"      # 输出纯文本

# =========================================================
# 🧩 emoji 统一处理
# =========================================================
emoji_pattern = re.compile(
    "[" 
    "\U0001F600-\U0001F64F"
    "\U0001F300-\U0001F5FF"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F780-\U0001F7FF"
    "\U0001F800-\U0001F8FF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FA6F"
    "\U0001FA70-\U0001FAFF"
    "\U00002700-\U000027BF"
    "\U0001F1E0-\U0001F1FF"
    "]+",
    flags=re.UNICODE
)

def replace_emojis(s: str) -> str:
    """将 emoji 转为可读名称，重复连续的只保留一个"""
    if not isinstance(s, str):
        return s
    prev = {"last": None}

    def repl(match):
        em = match.group(0)[0]
        name = emoji.demojize(em, language="zh")
        name = re.sub(r'^:+|:+$', '', name)
        name = re.sub(r'[_]+', '', name).strip()
        if not name or name == em:
            name = emoji.demojize(em)
            name = re.sub(r'^:+|:+$', '', name)
            name = re.sub(r'[_]+', '', name).strip()
        if not name:
            return ''
        if name == prev["last"]:
            return ''
        prev["last"] = name
        return name

    out = re.sub(emoji_pattern, repl, s)
    out = re.sub(r'\s+', ' ', out).strip()
    return out

# =========================================================
# 🧹 文本清洗
# =========================================================
def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return s

    # 基础清理
    s = s.replace("\\n", "\n")
    s = re.sub(r'[\b\r\t]', '', s)
    s = re.sub(r'\\"+', '"', s)
    s = re.sub(r'""+', '', s)
    s = re.sub(r';{2,}', ';', s)

    # 处理 emoji
    s = replace_emojis(s)

    # 删除独立“图片”或“图片图片”行
    s = re.sub(r'^\s*图片\s*图片?\s*$', '', s, flags=re.MULTILINE)

    # 删除乱码，只保留常规文字、标点、数字
    s = re.sub(r'[^\u4e00-\u9fa5A-Za-z0-9，。、“”‘’！；：：《》〈〉·,.!?()（）\s-]', '', s)

    # 合并多个空格为一个
    s = re.sub(r'\s+', ' ', s)
    # 去掉标点前后的空格
    s = re.sub(r'\s*([，。！？：；,.!?()（）])\s*', r'\1', s)
    # 去掉开头和结尾多余空格
    s = s.strip()

    return s

# =========================================================
# 🧹 清理列表元素（配料 / 步骤）
# =========================================================
def clean_list_item(x: str) -> str:
    if not isinstance(x, str):
        return x

    x = clean_text(x)

    # 删除无效内容
    if "成品" in x.replace(" ", "") or "看图文中的做法" in x.replace(" ", ""):
        return ""
    return x

# =========================================================
# 🔥 流式处理大文件
# =========================================================
with open(INPUT_FILE, "r", encoding="utf-8") as fin, \
     open(OUTPUT_TEXT_FILE, "w", encoding="utf-8") as fout:

    count = 0

    for line in fin:
        line = line.strip()
        if not line:
            continue

        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"❌ JSON解析失败：{e} | 行片段: {line[:80]}")
            continue

        # 删除 author
        obj.pop("author", None)

        # 清洗所有字段
        for k, v in obj.items():
            if isinstance(v, str):
                obj[k] = clean_text(v)
            elif isinstance(v, list):
                new_list = [clean_list_item(x) for x in v if clean_list_item(x)]
                obj[k] = new_list

        # dish 兜底
        dish = (obj.get("dish") or "").strip()
        if not dish or dish.lower() == "unknown":
            obj["dish"] = obj.get("name", "").strip()

        # 构建文本块
        desc = (obj.get("description") or "").strip()
        desc_block = f"描述：{desc}\n\n" if desc else ""
        ingredients = "\n".join(f"- {x}" for x in obj.get("recipeIngredient", []))

        steps_list = obj.get("recipeInstructions", [])
        steps = ""
        if steps_list:
            steps_lines = [f"{i+1}. {x}" for i, x in enumerate(steps_list)]
            steps = "步骤：\n" + "\n".join(steps_lines) + "\n\n"

        keywords = "，".join(obj.get("keywords", []))

        text_block = (
            f"菜名：{obj.get('name', '')}\n"
            f"类别：{obj.get('dish', '')}\n\n"
            f"{desc_block}"
            f"配料：\n{ingredients}\n\n"
            f"{steps}"
            f"关键词：{keywords}\n"
            + "=" * 60 + "\n\n"
        )

        # 删除多余空行
        text_block = re.sub(r'\n{2,}', '\n\n', text_block)

        fout.write(text_block)
        count += 1

        if count % 100000 == 0:
            print(f"已处理 {count} 行...")

print(f"🎉 完成！共处理 {count} 条记录。输出文件：{OUTPUT_TEXT_FILE}")


已处理 100000 行...
已处理 200000 行...
已处理 300000 行...
已处理 400000 行...
已处理 500000 行...
已处理 600000 行...
已处理 700000 行...
已处理 800000 行...
已处理 900000 行...
已处理 1000000 行...
已处理 1100000 行...
已处理 1200000 行...
已处理 1300000 行...
已处理 1400000 行...
已处理 1500000 行...
🎉 完成！共处理 1550151 条记录。输出文件：recipes_text.txt
